In [1]:
#dataset format: ori_data, neighbor_matrices, _, num_observed_vehicles
from feeder_ngsim import NgsimFeeder
from models import SpatialTransformerRNN
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [75]:
def preprocess_data(data, rescale_xy, neighbor_matrices, observed_last):
    # data: (N, C, T, V)
    # vehicle_ids = data[:, 1:2, :, :] # (N, 1, T, V)
    feature_id = [3, 4, 5, 6] # local x, local y, lane_id, mark
    ori_data = data[:, feature_id].detach()
    data = ori_data.detach().clone()

    new_mask = (data[:, :2, 1:] != 0) * (data[:, :2, :-1] != 0) # (N, 2, T-1, V)
    new_mask[:, :2, observed_last-1: observed_last+1] = 1 # the first is the target vehicle and the last observed and the first predicted frames' mask of this vehicle must be 1.
    # It is easier to predict the velocity of an object than predicting its location.
    # Calculate velocities p^{t+1} - p^{t} before feeding the data into the model.

    #predict velocity
    data[:, :2, 1:] = (data[:, :2, 1:] - data[:, :2, :-1]).float() * new_mask.float()
    data[:, :2, 0] = 0

    data = torch.cat([data[:, :2], ori_data[:, :2]], dim=1) # concat velocity and origin location.
    # print(data[:,0,:,:].max())
    # print(data[:,1,:,:].max())
    # print(data[:,2,:,:].max())
    # print(data[:,3,:,:].max())

    data = data.float().to(device)
    data[:, :4] = data[:, :4] / rescale_xy
    ori_data = ori_data.float().to(device)
    # vehicle_ids = vehicle_ids.long().to(args.device)

    A = neighbor_matrices.to(device)


    return data, ori_data, A

In [76]:
def preprocess_data2(data, neighbor_matrices, observed_last):
    # data: (N, C, T, V)
    # vehicle_ids = data[:, 1:2, :, :] # (N, 1, T, V)
    feature_id = [3, 4, 5, 6] # local x, local y, lane_id, mark
    ori_data = data[:, feature_id].detach()
    data = ori_data.detach().clone()


    new_mask = (data[:, :2, 1:] != 0) * (data[:, :2, :-1] != 0) # (N, 2, T-1, V)
    new_mask[:, :2, observed_last-1: observed_last+1] = 1 # the first is the target vehicle and the last observed and the first predicted frames' mask of this vehicle must be 1.
    # It is easier to predict the velocity of an object than predicting its location.
    # Calculate velocities p^{t+1} - p^{t} before feeding the data into the model.

    #predict velocity
    data[:, :2, 1:] = ((41/8)*(data[:, :2, 1:] - data[:, :2, :-1])).float() #* new_mask.float()
    data[:, :2, 0] = 0

    data = torch.cat([data[:, :2], ori_data[:, :2]], dim=1) # concat velocity and origin location.
    # print(data[:,0,:,:].max())
    # print(data[:,1,:,:].max())
    # print(data[:,2,:,:].max())
    # print(data[:,3,:,:].max())

    data = data.float().to(device)
    data[:, :4] = data[:, :4]
    ori_data = ori_data.float().to(device)
    # vehicle_ids = vehicle_ids.long().to(args.device)

    A = neighbor_matrices.to(device)


    return data, ori_data, A

In [77]:
max_x_velocity = 14.85
max_x = 36.155
max_y_velocity = 67.58
max_y = 486.76
th = 30
ds = 2





In [78]:
rescale_xy = torch.ones((1, 4, 1, 1)).to(device)
rescale_xy[:, 0] = max_x_velocity
rescale_xy[:, 1] = max_y_velocity
rescale_xy[:, 2] = max_x
rescale_xy[:, 3] = max_y

In [91]:
train_path = r'C:\Users\Hailong\Projects\ngsim_preprocessed_oldgit\TrainSet.mat'

dataset = NgsimFeeder(train_path, train_val_test='train')
loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=1)
input = torch.zeros([1,4,16,120])
outlabel = torch.zeros([1,2,25,120])

for iteration, (ori_data, neighbor_matrices, _, num_observed_vehicles) in enumerate(loader):
    now_history_frames = th // ds + 1 # 30 // 2 + 1, 30: history frames, 2: down sampling steps
    data, no_norm_loc_data, neighbor_matrices = preprocess_data(ori_data, rescale_xy, neighbor_matrices, observed_last=now_history_frames-1)
    input_data = data[:, :, :now_history_frames, :]
    # print(no_norm_loc_data[:,0,:,:].max())
    # print(input_data.shape)
    input = input_data
    #extract label to compare prediction with
    outputdata, _, _ = preprocess_data(ori_data, rescale_xy,neighbor_matrices, observed_last=now_history_frames-1)
    output_label = outputdata[:, :2, now_history_frames:, :]
    outlabel = output_label
    #print(outlabel)
    break




#input_data = data[:, :, :now_history_frames, :]



torch.Size([1, 4, 41, 120])
torch.Size([1, 4, 41, 120])


In [92]:
import timeit

model = SpatialTransformerRNN(
        in_size=4, out_size=2, seq2seq_type='gru',
        n_layers=4,
        interact_in_decoding=True,
        spatial_interact=True
    )
model.eval()
with torch.no_grad():
    checkpoint_fpath = "outputs/saved_models/STransformerRNN_1_4_1_gru_train_plus_val_0.0001_0.0_0.6_7_74036.pt"
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    A=torch.zeros([1,16,120,120],device=device)


    predicted = model(input, pra_A=A, pra_pred_length=25, input_mask=None, teacher_forcing_ratio=0.0,
                    pra_teacher_location=None, is_train=False)
    predicted = predicted[0]





In [93]:

predicted = predicted#*rescale_xy[:,:2,:,:]

#print((predicted*rescale_xy[:,:2,:,:]).max())
print(predicted)

tensor([[[[ 3.0125e-04,  1.7742e-04, -2.9840e-05,  ..., -2.9840e-05,
           -2.9840e-05, -2.9840e-05],
          [ 1.8373e-04,  1.0122e-04, -5.3657e-06,  ..., -5.3657e-06,
           -5.3657e-06, -5.3657e-06],
          [ 1.6451e-04,  7.3232e-05, -7.1679e-06,  ..., -7.1679e-06,
           -7.1679e-06, -7.1679e-06],
          ...,
          [ 2.0297e-05,  1.8536e-05,  4.7565e-06,  ...,  4.7565e-06,
            4.7565e-06,  4.7565e-06],
          [ 1.8899e-05,  1.7922e-05,  5.4828e-06,  ...,  5.4828e-06,
            5.4828e-06,  5.4828e-06],
          [ 1.7595e-05,  1.7323e-05,  6.1848e-06,  ...,  6.1848e-06,
            6.1848e-06,  6.1848e-06]],

         [[ 9.8128e-02,  5.8720e-02, -7.0543e-03,  ..., -7.0543e-03,
           -7.0543e-03, -7.0543e-03],
          [ 6.0752e-02,  3.4576e-02,  7.0228e-04,  ...,  7.0228e-04,
            7.0228e-04,  7.0228e-04],
          [ 5.4679e-02,  2.5714e-02,  2.2364e-04,  ...,  2.2364e-04,
            2.2364e-04,  2.2364e-04],
          ...,
     

In [94]:


x2y2 = torch.sum(torch.abs(predicted - outlabel) ** 2, dim=1) # x^2+y^2, (N, C, T, V)->(N, T, V)=(N, 25, 255)
overall_sum_time = x2y2.sum(dim=-1) # (N, T, V) -> (N, T)=(N, 25)
print(overall_sum_time)

tensor([[0.0161, 0.0096, 0.0111, 0.0123, 0.0123, 0.0122, 0.0139, 0.0145, 0.0141,
         0.0150, 0.0146, 0.0166, 0.0183, 0.0195, 0.0200, 0.0217, 0.0225, 0.0220,
         0.0211, 0.0222, 0.0237, 0.0254, 0.0264, 0.0266, 0.0272]],
       device='cuda:0')


In [154]:
#understand rescaling and the units
#write the function

rescale_xy[:,:2]

tensor([[[[14.8500]],

         [[67.5800]]]], device='cuda:0')

In [ ]:
def traj_pred(A, past_traj, model):


In [42]:
def preprocess_data(data, rescale_xy, neighbor_matrices, observed_last):
    # data: (N, C, T, V)
    # vehicle_ids = data[:, 1:2, :, :] # (N, 1, T, V)
    feature_id = [3, 4, 5, 6] # local x, local y, lane_id, mark
    ori_data = data[:, feature_id].detach()

    # convert to meters
    # ori_data = 0.3048*ori_data

    data = ori_data.detach().clone()


    new_mask = (data[:, :2, 1:] != 0) * (data[:, :2, :-1] != 0) # (N, 2, T-1, V)
    new_mask[:, :2, observed_last-1: observed_last+1, 0] = 1 # the first is the target vehicle and the last observed and the first predicted frames' mask of this vehicle must be 1.
    # It is easier to predict the velocity of an object than predicting its location.
    # Calculate velocities p^{t+1} - p^{t} before feeding the data into the model.

    data[:, :2, 1:] = (data[:, :2, 1:] - data[:, :2, :-1]).float() * new_mask.float()

    data[:, :2, 0] = 0

    data = torch.cat([data[:, :2], ori_data[:, :2]], dim=1) # concat velocity and origin location.



    data = data.float().to(device)
    data[:, :4] = data[:, :4] / rescale_xy
    ori_data = ori_data.float().to(device)
    # vehicle_ids = vehicle_ids.long().to(args.device)

    A = neighbor_matrices.to(device)


    return data, ori_data, A

In [43]:
def preprocess_data2(data, neighbor_matrices, observed_last):
    # data: (N, C, T, V)
    # vehicle_ids = data[:, 1:2, :, :] # (N, 1, T, V)
    feature_id = [3, 4, 5, 6] # local x, local y, lane_id, mark
    ori_data = data[:, feature_id].detach()

    # convert to meters
    # ori_data = 0.3048*ori_data

    data = ori_data.detach().clone()


    new_mask = (data[:, :2, 1:] != 0) * (data[:, :2, :-1] != 0) # (N, 2, T-1, V)
    new_mask[:, :2, observed_last-1: observed_last+1, 0] = 1 # the first is the target vehicle and the last observed and the first predicted frames' mask of this vehicle must be 1.
    # It is easier to predict the velocity of an object than predicting its location.
    # Calculate velocities p^{t+1} - p^{t} before feeding the data into the model.

    data[:, :2, 1:] = (data[:, :2, 1:] - data[:, :2, :-1]).float() * new_mask.float()

    data[:, :2, 0] = 0

    data = torch.cat([data[:, :2], ori_data[:, :2]], dim=1) # concat velocity and origin location.



    data = data.float().to(device)

    ori_data = ori_data.float().to(device)
    # vehicle_ids = vehicle_ids.long().to(args.device)

    A = neighbor_matrices.to(device)


    return data, ori_data, A

In [44]:
max_x_velocity = 14.85
max_x = 36.155
max_y_velocity = 67.58
max_y = 486.76
th = 30
ds = 2

In [46]:
rescale_xy = torch.ones((1, 4, 1, 1)).to(device)
rescale_xy[:, 0] = max_x_velocity
rescale_xy[:, 1] = max_y_velocity
rescale_xy[:, 2] = max_x
rescale_xy[:, 3] = max_y

In [56]:
train_path = r'C:\Users\Hailong\Projects\ngsim_preprocessed_oldgit\TrainSet.mat'

dataset = NgsimFeeder(train_path, train_val_test='train')
loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=1)
input = torch.zeros([1,4,16,120])
outlabel = torch.zeros([1,2,25,120])
A=torch.zeros([1,2,120,120])

for iteration, (ori_data, neighbor_matrices, _, num_observed_vehicles) in enumerate(loader):
    now_history_frames = th // ds + 1 # 30 // 2 + 1, 30: history frames, 2: down sampling steps
    data, no_norm_loc_data, neighbor_matrices = preprocess_data(ori_data, rescale_xy, neighbor_matrices, observed_last=now_history_frames-1)
    input_data = data[:, :, :now_history_frames, :]
    # print(no_norm_loc_data[:,0,:,:].max())
    # print(input_data.shape)
    input = input_data
    #extract label to compare prediction with
    outputdata, _, _ = preprocess_data2(ori_data, neighbor_matrices=neighbor_matrices, observed_last=now_history_frames-1)
    output_label = outputdata[:, 2:4, now_history_frames:, :]
    outlabel = output_label
    A=neighbor_matrices


    break

tensor([[[[1, 0, 0,  ..., 0, 0, 0],
          [0, 1, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[1, 0, 0,  ..., 0, 0, 0],
          [0, 1, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[1, 0, 0,  ..., 0, 0, 0],
          [0, 1, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         ...,

         [[1, 1, 0,  ..., 0, 0, 0],
          [1, 1, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[1, 1, 0,  ..., 0, 0, 0],
          [1, 1, 

In [57]:
model = SpatialTransformerRNN(
        in_size=4, out_size=2, seq2seq_type='gru',
        n_layers=4,
        interact_in_decoding=True,
        spatial_interact=True
    )
model.eval()
with torch.no_grad():
    checkpoint_fpath = "outputs/saved_models/STransformerRNN_1_4_1_gru_train_plus_val_0.0001_0.0_0.6_3_111054.pt"
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)


    predicted = model(input, pra_A=A, pra_pred_length=25, input_mask=None, teacher_forcing_ratio=0.0,
                    pra_teacher_location=None, is_train=False)
    predicted = predicted[0]

In [58]:

predicted = predicted*rescale_xy[:,2:4,:,:]

#print((predicted*rescale_xy[:,:2,:,:]).max())
print(predicted.shape)

torch.Size([1, 2, 25, 120])


In [60]:

x2y2 = torch.sum(torch.abs(predicted - outlabel) ** 2, dim=1) # x^2+y^2, (N, C, T, V)->(N, T, V)=(N, 25, 255)
overall_sum_time = x2y2.sum(dim=-1) # (N, T, V) -> (N, T)=(N, 25)
print(overall_sum_time)

tensor([[39207.4141, 17792.8145, 16384.8418, 15177.2773, 13957.9160, 12360.0137,
         10430.7480,  8344.1387,  6276.8760,  4357.1230,  2702.5264,  1412.9247,
           573.5718,   269.0222,   604.4550,  1690.0940,  3639.6899,  6574.7915,
         10633.3086, 15953.3359, 22669.8125, 30956.1699, 41004.1406, 53006.8398,
         67174.4219]], device='cuda:0')


In [74]:
(predicted-outlabel)[:,1,18].max()

tensor(14.4039, device='cuda:0')